In [ ]:
import pandas as pd
df = pd.read_csv('/content/cleaned_dataset.csv')
df.head()

,road_name,Start_point,End_point,delay_min,speed_kmh,duration_in_traffic_min,distance_km,traffic_condition
0,3rd Roundabout,"(24.6652373, 46.7968369)","(24.6631823, 46.7870053)",0.020370,48.384365,7.570370,6.094000,Blue
1,AL Hadad,"(24.6927368, 46.7944167)","(24.6437154, 46.7883665)",-0.887037,43.452083,15.818519,11.437333,Blue
2,Abdulaziz Al Fereh,"(24.6746147, 46.73819719999999)","(24.586689, 46.6373493)",0.959259,33.720841,36.285185,19.598889,Blue
3,Abdullah Bin Omar,"(24.750219, 46.8446611)","(24.7078712, 46.815598)",-0.705556,38.380332,20.425926,13.084222,Blue
4,Abi Hasain Al Hanbali,"(24.8022123, 46.6638953)","(24.7837499, 46.6243831)",0.096296,34.614967,15.612963,8.824000,Blue


In [ ]:
df.shape

(309, 8)

In [ ]:
# # Import necessary libraries
# from sklearn.ensemble import IsolationForest
# import pandas as pd

# # Load the dataset
# file_path = '/content/cleaned_dataset.csv'  # Replace with your actual file path
# data = pd.read_csv(file_path)

# # Split 'Start_point' and 'End_point' into two separate columns for latitude and longitude
# # Assuming the format is '(latitude, longitude)' for each coordinate
# data[['Start_lat', 'Start_lon']] = data['Start_point'].str.strip('()').str.split(',', expand=True).astype(float)
# data[['End_lat', 'End_lon']] = data['End_point'].str.strip('()').str.split(',', expand=True).astype(float)

# # Select the newly created numeric columns for anomaly detection
# columns_for_anomaly_detection = ['Start_lat', 'Start_lon', 'End_lat', 'End_lon']
# numeric_data = data[columns_for_anomaly_detection]

# # Fit the Isolation Forest model to detect anomalies
# iso_forest = IsolationForest(contamination=0.5, random_state=42)
# data['anomaly_score'] = iso_forest.fit_predict(numeric_data)

# # The 'anomaly_score' will be 1 for normal points and -1 for outliers
# # Keep only the normal data (where anomaly_score = 1)
# normal_data = data[data['anomaly_score'] == 1]

# # Select the top 100 rows from the normal data
# reduced_data = normal_data.head(100)

# # Save the reduced dataset to a CSV file
# #reduced_data.to_csv('reduced_data.csv', index=False)

# # Display the reduced dataset
# df = reduced_data


In [ ]:
import pandas as pd
from geopy.distance import geodesic

# Load the dataset
file_path = '/content/cleaned_dataset.csv'  # Replace with your actual file path
data = pd.read_csv(file_path)

# Convert 'Start_point' and 'End_point' into separate latitude and longitude columns
data[['Start_lat', 'Start_lon']] = data['Start_point'].str.strip('()').str.split(',', expand=True).astype(float)
data[['End_lat', 'End_lon']] = data['End_point'].str.strip('()').str.split(',', expand=True).astype(float)

# Sort data by the 'distance_km' column to prioritize larger distances
data_sorted = data.sort_values(by='distance_km', ascending=False)

# Define a function to calculate the geographical distance between two points
def calculate_geodesic_distance(row1, row2):
    start1 = (row1['Start_lat'], row1['Start_lon'])
    end1 = (row1['End_lat'], row1['End_lon'])
    start2 = (row2['Start_lat'], row2['Start_lon'])
    end2 = (row2['End_lat'], row2['End_lon'])

    # Calculate the geodesic distance between the start and end points of two rows
    return geodesic(start1, start2).kilometers + geodesic(end1, end2).kilometers

# Filter the data to keep streets that are not geographically close to each other
filtered_streets = []
distance_threshold_km = 10  # You can adjust this threshold based on how far apart the streets should be

for i, row in data_sorted.iterrows():
    is_far_enough = True
    for selected_row in filtered_streets:
        if calculate_geodesic_distance(row, selected_row) < distance_threshold_km:
            is_far_enough = False
            break
    if is_far_enough:
        filtered_streets.append(row)
    if len(filtered_streets) == 100:  # Stop once we have 100 streets
        break

# Convert the filtered streets list to a DataFrame
filtered_data = pd.DataFrame(filtered_streets)

# Save the reduced dataset to a CSV file
#filtered_data.to_csv('filtered_streets.csv', index=False)

# Display the reduced dataset
df = filtered_data

In [ ]:
pd.set_option('display.max_rows', None)
df.shape

(60, 12)

In [ ]:
df['road_name'].values

array(['King Abdul Aziz Road', 'Northern Ring Road', 'Jeddah Road',
       'طريق أبي بكر الصديق', 'Dammam Road', 'طريق خزام',
       'King Salman Road', 'Khurais Road', 'Al Janadriyah', 'Olaya Road',
       'Airport Road', 'East Ring Road', 'Western Ring Rd',
       'شارع المدينة المنورة', 'Al Thoumamah Rd', 'الشرق',
       'شارع شبه الجزيرة', 'طريق الجنادرية', 'وادي لبن',
       'أبي نصر الفارابي', 'King Abdullah Road', 'جبل طويق',
       'شارع وادي حنيفة', 'خالد ابن الوليد', 'التخصصي', 'Imam Malik Road',
       'طريق الأمير سعد بن عبدالرحمن الأول', 'Al Muhareb', 'العوفي',
       'العزايزه', 'Othman Bin Affan Road', 'Al Nahdha road',
       'طريق العروبة', 'Irqah', 'الضوء', 'الدائري الثاني', 'المعيزيلة',
       'رقوان', 'Al Naqeel', 'Yanbu', 'الأمير سعود بن عبدالله بن جلوي',
       'الحيازين', 'Al Nadwah', 'عثمان الضبي', 'ممر قرطبة', 'بنها',
       'طريق الامير مساعد بن جلوي', 'المهندس مساعد العنقري', 'Dhalam',
       'Al Jabour', 'الملد', 'سامطة', 'تقي الدين التميمي',
       'FAHHAD 

In [ ]:
# # prompt: drop row number 105

# # Assuming df is your DataFrame
# df = df.drop(105)


In [ ]:
# df

In [ ]:
import pandas as pd
from geopy.distance import geodesic

# Load the dataset
file_path = '/content/cleaned_dataset.csv'  # Replace with your actual file path
data = pd.read_csv(file_path)

# Convert 'Start_point' and 'End_point' into separate latitude and longitude columns
data[['Start_lat', 'Start_lon']] = data['Start_point'].str.strip('()').str.split(',', expand=True).astype(float)
data[['End_lat', 'End_lon']] = data['End_point'].str.strip('()').str.split(',', expand=True).astype(float)

# Sort data by the 'distance_km' column to prioritize larger distances
data_sorted = data.sort_values(by='distance_km', ascending=False)

# Define a function to calculate the geographical distance between two points
def calculate_geodesic_distance(row1, row2):
    start1 = (row1['Start_lat'], row1['Start_lon'])
    end1 = (row1['End_lat'], row1['End_lon'])
    start2 = (row2['Start_lat'], row2['Start_lon'])
    end2 = (row2['End_lat'], row2['End_lon'])

    # Calculate the geodesic distance between the start and end points of two rows
    return geodesic(start1, start2).kilometers + geodesic(end1, end2).kilometers

# Start with a larger threshold and reduce it gradually if needed
distance_threshold_km = 10  # Initial threshold (adjustable)
filtered_streets = []

while len(filtered_streets) < 100:
    temp_streets = []
    for i, row in data_sorted.iterrows():
        is_far_enough = True
        for selected_row in filtered_streets + temp_streets:
            if calculate_geodesic_distance(row, selected_row) < distance_threshold_km:
                is_far_enough = False
                break
        if is_far_enough:
            temp_streets.append(row)
        if len(filtered_streets + temp_streets) >= 100:
            break

    # Add the temp_streets to the final filtered streets
    filtered_streets.extend(temp_streets)

    # If we still don't have enough streets, reduce the distance threshold
    if len(filtered_streets) < 100:
        distance_threshold_km -= 1  # Reduce the threshold

    # Stop reducing if the threshold becomes too small
    if distance_threshold_km <= 0:
        break

# Convert the filtered streets list to a DataFrame
filtered_data = pd.DataFrame(filtered_streets)


# Display the reduced dataset
df= filtered_data


In [ ]:
df['road_name'].values

array(['King Abdul Aziz Road', 'Northern Ring Road', 'Jeddah Road',
       'طريق أبي بكر الصديق', 'Dammam Road', 'طريق خزام',
       'King Salman Road', 'Khurais Road', 'Al Janadriyah', 'Olaya Road',
       'Airport Road', 'East Ring Road', 'Western Ring Rd',
       'شارع المدينة المنورة', 'Al Thoumamah Rd', 'الشرق',
       'شارع شبه الجزيرة', 'طريق الجنادرية', 'وادي لبن',
       'أبي نصر الفارابي', 'King Abdullah Road', 'جبل طويق',
       'شارع وادي حنيفة', 'خالد ابن الوليد', 'التخصصي', 'Imam Malik Road',
       'طريق الأمير سعد بن عبدالرحمن الأول', 'Al Muhareb', 'العوفي',
       'العزايزه', 'Othman Bin Affan Road', 'Al Nahdha road',
       'طريق العروبة', 'Irqah', 'الضوء', 'الدائري الثاني', 'المعيزيلة',
       'رقوان', 'Al Naqeel', 'Yanbu', 'الأمير سعود بن عبدالله بن جلوي',
       'الحيازين', 'Al Nadwah', 'عثمان الضبي', 'ممر قرطبة', 'بنها',
       'طريق الامير مساعد بن جلوي', 'المهندس مساعد العنقري', 'Dhalam',
       'Al Jabour', 'الملد', 'سامطة', 'تقي الدين التميمي',
       'FAHHAD 

In [ ]:
df

,road_name,Start_point,End_point,delay_min,speed_kmh,duration_in_traffic_min,distance_km,traffic_condition,Start_lat,Start_lon,End_lat,End_lon
65,King Abdul Aziz Road,"(24.9996923, 46.7285093)","(24.6422095, 46.5499681)",1.774074,60.830829,57.201852,57.520333,Blue,24.999692,46.728509,24.642209,46.549968
78,Northern Ring Road,"(24.9560346, 47.01452949999999)","(24.7258755, 46.5830947)",2.801852,77.954628,43.085185,55.002000,Blue,24.956035,47.014529,24.725876,46.583095
62,Jeddah Road,"(24.6942703, 46.64157160000001)","(24.5263166, 46.4387046)",2.344444,75.920544,42.627778,52.827000,Blue,24.694270,46.641572,24.526317,46.438705
252,طريق أبي بكر الصديق,"(24.9673426, 46.7285093)","(24.6981028, 46.58851749999999)",1.016667,63.455807,48.105556,50.705000,Blue,24.967343,46.728509,24.698103,46.588517
52,Dammam Road,"(24.9568714, 47.0291162)","(24.7886798, 46.7188421)",0.666667,71.854651,39.600000,47.042000,Blue,24.956871,47.029116,24.788680,46.718842
265,طريق خزام,"(25.0520962, 46.8071244)","(24.8801496, 46.6994378)",-1.157407,63.251619,40.459259,42.592000,Blue,25.052096,46.807124,24.880150,46.699438
68,King Salman Road,"(24.97692050000003, 46.7237318)","(24.79621499999998, 46.5574488)",0.950000,68.242496,37.166667,42.181000,Blue,24.976921,46.723732,24.796215,46.557449
64,Khurais Road,"(24.946473, 46.9999471)","(24.7106093, 46.7623591)",3.503704,70.070459,35.637037,41.159000,Blue,24.946473,46.999947,24.710609,46.762359
24,Al Janadriyah,"(24.9445838, 46.9975312)","(24.7875365, 46.84589709999999)",-0.074074,73.288678,33.144444,40.178333,Blue,24.944584,46.997531,24.787537,46.845897
79,Olaya Road,"(24.8302126, 46.6995133)","(24.6639052, 46.612626)",2.688889,60.914283,38.455556,38.197000,Blue,24.830213,46.699513,24.663905,46.612626
